## MIMIC-III Querying Tutorial

[Tutorial is available online](https://mimic.physionet.org/tutorials/intro-to-mimic-iii/)

Dr. Maria P. Frushicheva @ MIT

In [40]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# used to calculate AUROC/accuracy
from sklearn import metrics

%matplotlib inline

In [41]:
# create a database connection
sqluser = 'mimic' 
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()

 ### Counting the Number of Patients in the Database

In [42]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
select count(subject_id)
from mimiciii.patients;
"""
data = pd.read_sql_query(query,con)
data

,count
0,46520


### Database Metadata for the Admissions Table

In [43]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT *
FROM admissions;
"""
data = pd.read_sql_query(query,con)
data.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_ioevents_data,has_chartevents_data
0,90,87,190659,2191-02-25 20:30:00,2191-04-25 15:18:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Private,,UNOBTAINABLE,,UNKNOWN/NOT SPECIFIED,NaT,NaT,NEWBORN,0,1,1
1,91,88,123010,2111-08-29 03:03:00,2111-09-03 14:24:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,,,,BLACK/AFRICAN AMERICAN,2111-08-29 01:44:00,2111-08-29 02:28:00,S/P MOTOR VEHICLE ACCIDENT-STABBING,0,1,1
2,92,89,188646,2185-06-17 05:22:00,2185-06-21 11:15:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,,UNOBTAINABLE,,UNKNOWN/NOT SPECIFIED,NaT,NaT,NEWBORN,0,1,1
3,93,91,121205,2177-04-23 00:08:00,2177-05-10 15:16:00,2177-05-10 15:16:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,,JEWISH,MARRIED,WHITE,2177-04-22 21:02:00,2177-04-23 04:03:00,FEVER,1,1,1
4,94,92,142807,2122-12-13 19:30:00,2123-03-04 13:47:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,SHORT TERM HOSPITAL,Medicaid,,UNOBTAINABLE,,WHITE,NaT,NaT,NEWBORN,0,1,1


### Database Metadata for the Patients Table

In [44]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT *
FROM patients;
"""
data = pd.read_sql_query(query,con)
data.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,612,646,M,2128-01-05,None,None,None,0
1,613,647,M,2106-03-24,None,None,None,0
2,614,648,M,2139-07-13,None,None,None,0
3,615,649,M,2177-06-23,None,None,None,0
4,616,650,M,2051-04-15,2111-12-28 00:00:00,None,2111-12-28 00:00:00,1


### Database Metadata for the CharteventsTable

In [45]:
## Too long to execute!!!
#cur.execute('SET search_path to ' + schema_name)
#query = \
#"""
#SELECT *
#FROM chartevents;
#"""
#data = pd.read_sql_query(query,con)
#data.head()

 ### The Number of Patients in the Patients Table

In [46]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT COUNT(*)
FROM patients;
"""
data = pd.read_sql_query(query,con)
data

,count
0,46520


### The Distinct Values of Gender

In [47]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT DISTINCT(gender)
FROM patients;
"""
data = pd.read_sql_query(query,con)
data

,gender
0,M
1,F


### The Number of Female Patients

In [48]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT COUNT(*)
FROM patients
WHERE gender = 'F';
"""
data = pd.read_sql_query(query,con)
data

,count
0,20399


### The Numbers of Male and Female Patients

In [49]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT gender, COUNT(*)
FROM patients
GROUP BY gender;
"""
data = pd.read_sql_query(query,con)
data

,gender,count
0,M,26121
1,F,20399


### Count the Number of Patients Who Died in the Hospital

In [50]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT expire_flag, COUNT(*)
FROM patients
GROUP BY expire_flag;
"""
data = pd.read_sql_query(query,con)
data

,expire_flag,count
0,0,30761
1,1,15759


### Patient Age and Mortality

Combine the patients and admissions tables to find patient admission dates and dates of birth. We have denoted ‘admissions’ with the alias ‘a’ and ‘patients’ with alias ‘p’.

In [51]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT p.subject_id, p.dob, a.hadm_id,
    a.admittime, p.expire_flag
FROM admissions a
INNER JOIN patients p
ON p.subject_id = a.subject_id;
"""
data = pd.read_sql_query(query,con)
data.shape

(58976, 5)

 The earliest admission time for each patient. 

In [52]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT p.subject_id, p.dob, a.hadm_id,
    a.admittime, p.expire_flag,
    MIN (a.admittime) OVER (PARTITION BY p.subject_id) AS first_admittime
FROM admissions a
INNER JOIN patients p
ON p.subject_id = a.subject_id
ORDER BY a.hadm_id, p.subject_id;
"""
data = pd.read_sql_query(query,con)
data.head()

,subject_id,dob,hadm_id,admittime,expire_flag,first_admittime
0,58526,2082-03-21,100001,2117-09-11 11:46:00,0,2117-09-11 11:46:00
1,54610,2090-05-19,100003,2150-04-17 15:34:00,1,2150-04-17 15:34:00
2,9895,2059-05-07,100006,2108-04-06 15:49:00,1,2108-04-06 15:49:00
3,23018,2071-06-04,100007,2145-03-31 05:33:00,0,2145-03-31 05:33:00
4,533,2101-07-30,100009,2162-05-16 15:56:00,0,2162-05-16 15:56:00


A patient’s age is given by the difference between their date of birth and the date of their first admission. 

In [53]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
WITH first_admission_time AS
(
  SELECT
      p.subject_id, p.dob, p.gender
      , MIN (a.admittime) AS first_admittime
      , MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) )
          AS first_admit_age
  FROM patients p
  INNER JOIN admissions a
  ON p.subject_id = a.subject_id
  GROUP BY p.subject_id, p.dob, p.gender
  ORDER BY p.subject_id
)
SELECT
    subject_id, dob, gender
    , first_admittime, first_admit_age
    , CASE
        -- all ages > 89 in the database were replaced with 300
        WHEN first_admit_age > 89
            then '>89'
        WHEN first_admit_age >= 14
            THEN 'adult'
        WHEN first_admit_age <= 1
            THEN 'neonate'
        ELSE 'middle'
        END AS age_group
FROM first_admission_time
ORDER BY subject_id
"""
data = pd.read_sql_query(query,con)
data.head()

,subject_id,dob,gender,first_admittime,first_admit_age,age_group
0,2,2138-07-17,M,2138-07-17 19:04:00,0.00,neonate
1,3,2025-04-11,M,2101-10-20 19:08:00,76.52,adult
2,4,2143-05-12,F,2191-03-16 00:28:00,47.84,adult
3,5,2103-02-02,M,2103-02-02 04:31:00,0.00,neonate
4,6,2109-06-21,F,2175-05-30 07:15:00,65.94,adult


### The number of adult patients, whether or not they died, and therefore, their mortality rate.

In [54]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
WITH first_admission_time AS
(
  SELECT
      p.subject_id, p.dob, p.gender
      , MIN (a.admittime) AS first_admittime
      , MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) )
          AS first_admit_age
  FROM patients p
  INNER JOIN admissions a
  ON p.subject_id = a.subject_id
  GROUP BY p.subject_id, p.dob, p.gender
  ORDER BY p.subject_id
)
, age as
(
  SELECT
      subject_id, dob, gender
      , first_admittime, first_admit_age
      , CASE
          -- all ages > 89 in the database were replaced with 300
          -- we check using > 100 as a conservative threshold to ensure we capture all these patients
          WHEN first_admit_age > 100
              then '>89'
          WHEN first_admit_age >= 14
              THEN 'adult'
          WHEN first_admit_age <= 1
              THEN 'neonate'
          ELSE 'middle'
          END AS age_group
  FROM first_admission_time
)
select age_group, gender
  , count(subject_id) as NumberOfPatients
from age
group by age_group, gender
"""
data = pd.read_sql_query(query,con)
data

,age_group,gender,numberofpatients
0,adult,M,21179
1,neonate,F,3629
2,>89,M,697
3,adult,F,15476
4,neonate,M,4245
5,>89,F,1294


### ICU Stays: Patient ICU movements are recorded in the Transfers table. 

Entries that have nothing in both previous and current careunit columns indicate that the patient has been transfered between non intensive care units.

In [55]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT *
FROM transfers;
"""
data = pd.read_sql_query(query,con)
print data.shape
data

(261897, 13)


,row_id,subject_id,hadm_id,icustay_id,dbsource,eventtype,prev_careunit,curr_careunit,prev_wardid,curr_wardid,intime,outtime,los
0,141,32,175413,NaN,carevue,transfer,,,7.0,2.0,2170-04-15 20:21:00,2170-04-23 12:45:00,184.40
1,142,32,175413,NaN,carevue,discharge,,,2.0,NaN,2170-04-23 12:45:00,NaT,NaN
2,143,33,176176,296681.0,carevue,admit,,MICU,NaN,12.0,2116-12-23 22:31:53,2116-12-25 11:49:55,37.30
3,144,33,176176,NaN,carevue,transfer,MICU,,12.0,45.0,2116-12-25 11:49:55,2116-12-27 12:05:48,48.26
4,145,33,176176,NaN,carevue,discharge,,,45.0,NaN,2116-12-27 12:05:48,NaT,NaN
5,146,34,115799,NaN,carevue,admit,,,NaN,17.0,2186-07-18 16:46:43,2186-07-18 18:10:49,1.40
6,147,34,115799,263086.0,carevue,transfer,,MICU,17.0,23.0,2186-07-18 18:10:49,2186-07-19 11:27:20,17.28
7,148,34,115799,NaN,carevue,transfer,MICU,,23.0,2.0,2186-07-19 11:27:20,2186-07-20 16:07:25,28.67
8,149,34,115799,NaN,carevue,discharge,,,2.0,NaN,2186-07-20 16:07:25,NaT,NaN
9,150,34,144319,290505.0,metavision,admit,,CCU,NaN,7.0,2191-02-23 05:25:32,2191-02-23 05:33:33,0.13


### Exercise 1

In [56]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, ie.intime, ie.outtime
FROM icustays AS ie;
"""
data = pd.read_sql_query(query,con)
print data.shape
data.head()

(61532, 5)


,subject_id,hadm_id,icustay_id,intime,outtime
0,109,139061,257358,2141-09-11 10:13:28,2141-09-12 16:53:07
1,109,172335,262652,2141-09-20 20:44:36,2141-09-22 21:44:50
2,109,126055,236124,2141-10-13 23:11:01,2141-10-25 20:49:04
3,109,125288,257134,2141-11-18 14:01:37,2141-11-19 21:35:18
4,109,161950,237552,2141-11-24 16:12:07,2141-11-26 21:51:50


### Exercise 2: Using the Patients table retrieve the calculated age of patients. 

In [57]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, ie.intime, ie.outtime,
    ROUND((cast(ie.intime as date) - cast(p.dob as date))/365.242, 2) AS age
FROM icustays ie
INNER JOIN patients p
ON ie.subject_id = p.subject_id;
"""
data = pd.read_sql_query(query,con)
print data.shape
data.head()

(61532, 6)


,subject_id,hadm_id,icustay_id,intime,outtime,age
0,109,139061,257358,2141-09-11 10:13:28,2141-09-12 16:53:07,24.10
1,109,172335,262652,2141-09-20 20:44:36,2141-09-22 21:44:50,24.12
2,109,126055,236124,2141-10-13 23:11:01,2141-10-25 20:49:04,24.18
3,109,125288,257134,2141-11-18 14:01:37,2141-11-19 21:35:18,24.28
4,109,161950,237552,2141-11-24 16:12:07,2141-11-26 21:51:50,24.30


### Exercise 3: Separate neonates from adult patients.

In [58]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
WITH first_time AS
(
SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, ie.intime, ie.outtime,
    ROUND((cast(ie.intime as date) - cast(p.dob as date))/365.242, 2) AS age
FROM icustays ie
INNER JOIN patients p
ON ie.subject_id = p.subject_id
)
SELECT
    subject_id, intime, age
    , CASE
        -- all ages > 89 in the database were replaced with 300
        WHEN age > 89
            then '>89'
        WHEN age >= 14
            THEN 'adult'
        WHEN age <= 1
            THEN 'neonate'
        ELSE 'middle'
        END AS age_group
FROM first_time
ORDER BY subject_id
"""
data = pd.read_sql_query(query,con)
print data.shape
data.head()

(61532, 4)


,subject_id,intime,age,age_group
0,2,2138-07-17 21:20:07,0.00,neonate
1,3,2101-10-20 19:10:11,76.52,adult
2,4,2191-03-16 00:29:31,47.84,adult
3,5,2103-02-02 06:04:24,0.00,neonate
4,6,2175-05-30 21:30:54,65.94,adult
